<a href="https://colab.research.google.com/github/GerardoFesta/DLforJITDefectPrediction/blob/main/src/10FoldCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder


In [ ]:
dataframe = pd.read_csv("data_scaled.csv")
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,2:26].astype(float)
Y = dataset[:,1]

In [ ]:
print(len(X))
print(len(Y))


193800
193800


In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [ ]:
from tensorflow.python.keras.backend  import set_session,clear_session,get_session
import gc

# funzione per liberare spazio ad ogni iterazione della cross-validation
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model 
    except:
        pass

    print("Reset Keras ",gc.collect()) 

    
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.compat.v1.Session(config=config))

In [ ]:

def create_model(grid):
	# create model
  model = Sequential()
  model.add(Dense(24, input_shape=(24,), activation='linear',kernel_regularizer=tf.keras.regularizers.l2(grid["decay"]),
   bias_regularizer=tf.keras.regularizers.l2(grid["decay"])))
  for x in range(grid["hidden_size"]):
    model.add(Dense(13, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2(grid["decay"])))
  model.add(Dense(1, activation='sigmoid'))
  # optimizer
  opt = SGD(
    learning_rate=grid["lr"], momentum=grid["momentum"]
  )
	# Compile model
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
import csv  
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

header = [ 'hidden_size', 'lr', "momentum", "decay", "hasEarlyStopped", "10FoldCV-Loss", "10FoldCV-Acc"]


with open('result.csv', 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)



In [ ]:
from sklearn.model_selection import ParameterGrid
# setto i valori dei parametri che voglio testare e creo tutte le possibili combinazioni
param_grid = {'hidden_size': [1, 2, 3],'lr': [0.01, 0.001, 0.0001], "momentum":[0.7, 0.8, 0.9], "decay":[0.1, 0.3, 0.5]}
expanded_grid = ParameterGrid(param_grid)

batch_size=1024

In [ ]:

res ={"hidden_size":[], "lr":[], "momentum":[], "decay":[], "EarlyStopped":[], "10FoldCV-Loss":[], "10FoldCV-Acc":[]}

for i in range(len(expanded_grid)):
        print(expanded_grid[i])
        fold_no=1
        loss_per_fold=[]
        acc_per_fold = []
        for train, test in skf.split(X, encoded_Y):
            model = create_model(expanded_grid[i])

            
            # Generate a print
            print('------------------------------------------------------------------------')
            print(f'Training for fold {fold_no} ...')
            #Create the callback for early stopping monitor. If the loss doesn't improve for more than 4 iterations, it stops the process
            callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6, verbose = 1)
            # Fit data to model
            history = model.fit(X[train], encoded_Y[train],
                      batch_size=batch_size,
                      epochs=150,
                      verbose=1,
                      steps_per_epoch=20,
                      callbacks = [callback])
            

            # Generate generalization metrics
            scores,acc = model.evaluate(X[test], encoded_Y[test], verbose=0)
            #print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
            loss_per_fold.append(scores)
            acc_per_fold.append(acc)
            # Increase fold number
            fold_no = fold_no + 1
            #reset_keras()
       
        res["hidden_size"].append(expanded_grid[i]["hidden_size"])
        res["lr"].append(expanded_grid[i]["lr"])
        res["momentum"].append(expanded_grid[i]["momentum"])
        res["decay"].append(expanded_grid[i]["decay"])
  
        res["EarlyStopped"].append(len(history.history['loss']))
        res["10FoldCV-Loss"].append(round(sum(loss_per_fold)/10,3))
        res["10FoldCV-Acc"].append(round(sum(acc_per_fold)/10,3))
        with open('result.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([res["hidden_size"][i],res["lr"][i],res["momentum"][i],res["decay"][i],res["EarlyStopped"][i],res["10FoldCV-Loss"][i],res["10FoldCV-Acc"][i]])
                print (i+1, "/", len(expanded_grid))


In [ ]:
with open('result.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([res["hidden_size"][0],res["lr"][0],res["momentum"][0],res["decay"][0],res["10FoldCV-Loss"][0]])
                print (i+1, "/", len(expanded_grid))

  

81 / 81
